In [ ]:
import tkinter as tk
from tkinter import messagebox


class StockNode:
    def __init__(self, ticker, price, quantity, color=False):
        self.ticker = ticker
        self.price = price
        self.quantity = quantity
        self.color = color 
        self.left = None
        self.right = None
        self.parent = None


class RedBlackTree:
    def __init__(self):
        self.TNULL = StockNode("", 0, 0, color=True)
        self.root = self.TNULL

    def rotate_left(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def rotate_right(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x
        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    def fix_insert(self, k):
        while k.parent and not k.parent.color:
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left
                if u and not u.color:
                    u.color = True
                    k.parent.color = True
                    k.parent.parent.color = False
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        k = k.parent
                        self.rotate_right(k)
                    k.parent.color = True
                    k.parent.parent.color = False
                    self.rotate_left(k.parent.parent)
            else:
                u = k.parent.parent.right
                if u and not u.color:
                    u.color = True
                    k.parent.color = True
                    k.parent.parent.color = False
                    k = k.parent.parent
                else:
                    if k == k.parent.right:
                        k = k.parent
                        self.rotate_left(k)
                    k.parent.color = True
                    k.parent.parent.color = False
                    self.rotate_right(k.parent.parent)
            if k == self.root:
                break
        self.root.color = True

    def insert(self, ticker, price, quantity):
        node = StockNode(ticker, price, quantity)
        node.left = node.right = self.TNULL

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            x = x.left if node.ticker < x.ticker else x.right

        node.parent = y
        if y is None:
            self.root = node
        elif node.ticker < y.ticker:
            y.left = node
        else:
            y.right = node

        if node.parent is None:
            node.color = True
            return

        if node.parent.parent is None:
            return

        self.fix_insert(node)

    def in_order_helper(self, node, result):
        if node != self.TNULL:
            self.in_order_helper(node.left, result)
            result.append(f"[{node.ticker}] Price: {node.price}, Qty: {node.quantity}")
            self.in_order_helper(node.right, result)

    def display_portfolio(self):
        result = []
        self.in_order_helper(self.root, result)
        return "\n".join(result)

    def search_helper(self, node, key):
        if node == self.TNULL or node.ticker == key:
            return node
        if key < node.ticker:
            return self.search_helper(node.left, key)
        return self.search_helper(node.right, key)

    def search_stock(self, key):
        result = self.search_helper(self.root, key)
        return result if result != self.TNULL else None

    def transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def delete_node(self, node, key):
        if node == self.TNULL:
            return
        if key < node.ticker:
            self.delete_node(node.left, key)
        elif key > node.ticker:
            self.delete_node(node.right, key)
        else:
            self.transplant(node, node.right if node.right else self.TNULL)
            del node

    def delete_stock(self, key):
        self.delete_node(self.root, key)

    def total_value_helper(self, node):
        if node == self.TNULL:
            return 0
        return self.total_value_helper(node.left) + (node.price * node.quantity) + self.total_value_helper(node.right)

    def total_portfolio_value(self):
        return self.total_value_helper(self.root)




class StockApp:
    def __init__(self, root):
        self.tree = RedBlackTree()
        self.root = root
        root.title("Stock Portfolio Manager")
        root.geometry("550x500")

        tk.Label(root, text="Ticker:").grid(row=0, column=0, sticky="e")
        tk.Label(root, text="Price:").grid(row=1, column=0, sticky="e")
        tk.Label(root, text="Quantity:").grid(row=2, column=0, sticky="e")

        self.ticker_entry = tk.Entry(root)
        self.price_entry = tk.Entry(root)
        self.quantity_entry = tk.Entry(root)

        self.ticker_entry.grid(row=0, column=1)
        self.price_entry.grid(row=1, column=1)
        self.quantity_entry.grid(row=2, column=1)

        
        tk.Button(root, text="Add Stock", command=self.add_stock).grid(row=3, column=0, pady=5)
        tk.Button(root, text="Display Portfolio", command=self.display_portfolio).grid(row=3, column=1)
        tk.Button(root, text="Search Stock", command=self.search_stock).grid(row=4, column=0)
        tk.Button(root, text="Delete Stock", command=self.delete_stock).grid(row=4, column=1)
        tk.Button(root, text="Total Value", command=self.show_total_value).grid(row=5, column=0, columnspan=2, pady=5)

    
        self.output = tk.Text(root, height=15, width=65)
        self.output.grid(row=6, column=0, columnspan=2, padx=10)

    def add_stock(self):
        try:
            ticker = self.ticker_entry.get()
            price = float(self.price_entry.get())
            quantity = int(self.quantity_entry.get())
            self.tree.insert(ticker, price, quantity)
            self.output.insert(tk.END, f"Added: {ticker}\n")
        except ValueError:
            messagebox.showerror("Error", "Invalid price or quantity!")

    def display_portfolio(self):
        result = self.tree.display_portfolio()
        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, result if result else "Portfolio is empty.\n")

    def search_stock(self):
        ticker = self.ticker_entry.get()
        stock = self.tree.search_stock(ticker)
        if stock:
            self.output.insert(tk.END, f"Found: {stock.ticker} | Price: ₹{stock.price} | Qty: {stock.quantity}\n")
        else:
            self.output.insert(tk.END, "Stock not found.\n")

    def delete_stock(self):
        ticker = self.ticker_entry.get()
        self.tree.delete_stock(ticker)
        self.output.insert(tk.END, f"Deleted {ticker} if existed.\n")

    def show_total_value(self):
        total = self.tree.total_portfolio_value()
        self.output.insert(tk.END, f"\nTotal Portfolio Value: ₹{total:.2f}\n")

if __name__ == "__main__":
    window = tk.Tk()
    app = StockApp(window)
    window.mainloop()

